In [ ]:
# config
config = {
    "data_path": "../",
    "model": {
        "loss_smooth": 1.0,
        "optimizer_params": {"lr": 0.001, "weight_decay": 0.0},
        "scheduler": {
            "name": "CosineAnnealingLR",
            "params": {
                "CosineAnnealingLR": {"T_max": 50, "eta_min": 1e-06, "last_epoch": -1},
                "ReduceLROnPlateau": {
                    "factor": 0.316,
                    "mode": "min",
                    "patience": 3,
                    "verbose": True,
                },
            },
        },
    },
    "output_dir": "models",
    "progress_bar_refresh_rate": 500,
    "seed": 42,
    "train_bs": 64,
    "trainer": {
        "enable_progress_bar": True,
        "max_epochs": 50,
        "min_epochs": 30,
        "accelerator": "mps",
        "devices": 1,
    },
    "valid_bs": 64,
    "workers": 0,
    "device": "mps",
    "folds": {
        "n_splits": 4,
        "random_state": 42,
        "train_folds": [0, 1, 2, 3]
    }
}

import pandas as pd

train = pd.read_csv('data/single_turbine_data/train_reduced_unskewed_extra.csv')
test = pd.read_csv('data/single_turbine_data/test_reduced_unskewed_extra.csv')

label = ['1_Gear oil temperature (°C)']

X_train = train.drop(label, axis=1)
y_train = train[label]
X_test = test.drop(label, axis=1)
y_test = test[label]

# convert to datetime
X_train['# Date and time'] = pd.to_datetime(X_train['# Date and time'])
X_test['# Date and time'] = pd.to_datetime(X_test['# Date and time'])

# Setting the index
X_train.set_index('# Date and time', inplace=True)
X_test.set_index('# Date and time', inplace=True)

original_cols = ['1_Wind direction (°)',
       '1_Nacelle position (°)', '1_Power (kW)',
       '1_Front bearing temperature (°C)', '1_Rear bearing temperature (°C)',
       '1_Stator temperature 1 (°C)', '1_Nacelle ambient temperature (°C)',
       '1_Nacelle temperature (°C)', '1_Transformer temperature (°C)',
       '1_Generator bearing rear temperature (°C)',
       '1_Generator bearing front temperature (°C)', '1_Temp. top box (°C)',
       '1_Hub temperature (°C)', '1_Ambient temperature (converter) (°C)',
       '1_Rotor bearing temp (°C)', '1_Transformer cell temperature (°C)', '1_Generator RPM (RPM)']
extras = ['month_sin', 'month_cos', 'hour_sin', 'hour_cos', 
'curtailed', 
'offline',
]
leadsnlags = [
       '1_Wind direction (°)_lead6', 
       '1_Nacelle position (°)_lead3',
       '1_Power (kW)_lag6', 
       '1_Stator temperature 1 (°C)',
       '1_Stator temperature 1 (°C)_lag1',
       '1_Nacelle ambient temperature (°C)_lead6',
       '1_Transformer temperature (°C)_lead6',
       '1_Generator bearing rear temperature (°C)_lag1',
       '1_Temp. top box (°C)_lag1', 
       '1_Hub temperature (°C)_lead6',
       '1_Ambient temperature (converter) (°C)_lead6',
       '1_Transformer cell temperature (°C)_lead6',
       '1_Generator RPM (RPM)_lead6']

cols = original_cols + extras + leadsnlags
X_test = X_test[cols]
X_train = X_train[cols]

# scale the data
from sklearn.preprocessing import StandardScaler, RobustScaler
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar
import lightning.pytorch as pl
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau

# Convert Pandas DataFrame to PyTorch Tensor
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

def quantile_loss(preds, targets, alpha=0.05):
    errors = targets - preds
    
    lower_quantile = torch.max((alpha - 1) * errors, alpha * errors).mean()
    upper_quantile = torch.max(((1 - alpha) - 1) * errors, (1 - alpha) * errors).mean()
    
    return lower_quantile + upper_quantile



# Define Neural Network
class LightningModule(pl.LightningModule):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.config = config
        self.layer1 = nn.Linear(input_dim, input_dim*2)
        self.layer2 = nn.Linear(input_dim*2, input_dim*2)
        self.layer3 = nn.Linear(input_dim*2, output_dim)
        # self.relu = nn.ReLU()
        self.relu = nn.LeakyReLU(0.3)
        # self.dropout = nn.Dropout(p=0.2)
        self.loss = nn.MSELoss()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        # x = self.dropout(x)
        x = self.relu(self.layer2(x))
        # x = self.dropout(x)
        x = self.layer3(x)
        return x

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters())

        if self.config['model']["scheduler"]["name"] == "CosineAnnealingLR":
            scheduler = CosineAnnealingLR(
                optimizer,
                **self.config['model']["scheduler"]["params"][self.config['model']["scheduler"]["name"]],
            )
            lr_scheduler_dict = {"scheduler": scheduler, "interval": "step"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler_dict}
        elif self.config['model']["scheduler"]["name"] == "ReduceLROnPlateau":
            scheduler = ReduceLROnPlateau(
                optimizer,
                **self.config['model']["scheduler"]["params"][self.config['model']["scheduler"]["name"]],
            )
            lr_scheduler = {"scheduler": scheduler, "monitor": "val_loss"}
            return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        for param_group in self.trainer.optimizers[0].param_groups:
            lr = param_group["lr"]
        self.log("lr", lr, on_step=False, on_epoch=True, prog_bar=True)
        logs = {"train_loss": loss, "lr": lr}
        return {"loss": loss, "log": logs}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        return {"val_loss": loss}


from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

# Prepare the KFold cross-validation
kf = KFold(n_splits=config["folds"]["n_splits"])

# Initialize results DataFrame
overall_results = pd.DataFrame()

# Main loop for K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train, y_train)):
    print(f"Training Fold {fold+1}")
    # Subset the data
    X_train_fold = X_train[train_index]
    y_train_fold = y_train[train_index]
    X_val_fold = X_train[val_index]
    y_val_fold = y_train[val_index]

    # Create DataLoader for this fold
    train_dataset = TensorDataset(torch.FloatTensor(X_train_fold), torch.FloatTensor(y_train_fold))
    train_loader = DataLoader(train_dataset, batch_size=config["train_bs"], shuffle=True)

    val_dataset = TensorDataset(torch.FloatTensor(X_val_fold), torch.FloatTensor(y_val_fold))
    val_loader = DataLoader(val_dataset, batch_size=config["valid_bs"], shuffle=True)

    # Initialize model
    model = LightningModule(X_train.shape[1], y_train.shape[1])

    early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1)
    progress_bar_callback = TQDMProgressBar(refresh_rate=config["progress_bar_refresh_rate"])

    checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_top_k=1,
    verbose=1,
    dirpath=config["output_dir"],
    filename="best_model_fold_" + str(fold + 1),
)
    
    # Initialize trainer
    trainer = pl.Trainer(
        callbacks=[early_stop_callback, progress_bar_callback, checkpoint_callback],
        logger=logger,
        **config["trainer"]
    )

    # Train the model on this fold
    trainer.fit(model, train_loader, val_loader)

    # Validation
    model.eval()
    y_pred_fold = model(torch.FloatTensor(X_val_fold)).detach().numpy()

    # Inverse scaling for validation data and prediction
    y_val_fold = scaler.inverse_transform(y_val_fold)
    y_pred_fold = scaler.inverse_transform(y_pred_fold)

    # Evaluate the model on this fold
    rmse_fold = sqrt(mean_squared_error(y_val_fold, y_pred_fold))
    print(f'Validation RMSE for Fold {fold+1}: {rmse_fold}')

    # Save results for this fold
    results_df = pd.DataFrame(
        {
            'Model': f'Neural Network - Fold {fold+1}',
            'Validation RMSE': rmse_fold,
            'Iterations': trainer.global_step,
            'Learning Rate': config['model']['optimizer_params']['lr'],
            'Depth': [X_train.shape[1], 64, 64, y_train.shape[1]],
            'Loss Function': 'MSE',
            'Features': ', '.join(cols),
        }
    )
    overall_results = pd.concat([overall_results, results_df])

    # save model
    # torch.save(model.state_dict(), f"models/nn_fold{fold+1}.pth")

from pathlib import Path
# Save results
results_file = Path('results.csv')
# Save overall results
if results_file.exists():
    existing_df = pd.read_csv(results_file)
    overall_results = pd.concat([existing_df, overall_results])

overall_results.to_csv(results_file, index=False)


In [2]:
# plt.scatter(y_test, y_pred)

In [3]:
# save the model
# torch.save(model.state_dict(), 'nn.pt')

In [15]:
import os
import numpy as np
from math import sqrt
from sklearn.metrics import mean_squared_error

# Initialize variables
combined_preds = []
path = 'models'

# unscale the data
# if the mean of y_test is < 10 
if y_test.mean() < 10:
    y_test = scaler.inverse_transform(y_test)

# Iterate through all models in the folder
for filename in os.listdir(path):
    if filename.endswith('v2.ckpt'):
        checkpoint_path = os.path.join(path, filename)
        
        state_dict = torch.load(checkpoint_path)
        model = LightningModule(X_train.shape[1], y_train.shape[1])
        model.load_state_dict(state_dict['state_dict'])
        
        # Switch to evaluation mode
        model.eval()
        
        # Perform inference
        with torch.no_grad():
            y_pred = model(X_test_tensor)
        
        # Process predictions
        y_pred = y_pred.detach().numpy()
        y_pred = scaler.inverse_transform(y_pred)
        
        # Store predictions for later
        combined_preds.append(y_pred)

# Combine all predictions (mean ensemble, you can also try other techniques)
# combined_preds = np.mean(np.array(combined_preds), axis=0)

# Calculate the combined RMSE
# combined_rmse = sqrt(mean_squared_error(y_test, combined_preds))
# print(f'Combined Test RMSE: {combined_rmse:.3f}')

In [17]:
import itertools

best_rmse = float('inf')
best_weights = None

# Generate weight combinations, summing to 1
num_models = len(combined_preds)
for weights in itertools.product(np.linspace(0, 1, 11), repeat=num_models):
    if sum(weights) == 1:
        
        # Calculate the weighted average
        combined_preds_weighted = np.average(np.array(combined_preds), axis=0, weights=weights)
        
        # Calculate RMSE for the weighted average
        rmse_weighted = sqrt(mean_squared_error(y_test, combined_preds_weighted))
        
        # Update the best RMSE and corresponding weights
        if rmse_weighted < best_rmse:
            best_rmse = rmse_weighted
            best_weights = weights

# Show the best RMSE and weights
print(f'Best Test RMSE: {best_rmse:.3f} with weights {best_weights}')


Best Test RMSE: 0.475 with weights (0.1, 0.0, 0.30000000000000004, 0.6000000000000001)
